# Basic Usage Tutorial

This notebook demonstrates the fundamental concepts and operations in Tellus. You'll learn how to:

- Create and manage simulations
- Configure storage locations
- Transfer files with progress tracking
- Use context-aware path resolution

## Setup

First, let's import the necessary modules and set up our environment:

In [ ]:
import os
import tempfile
from pathlib import Path

# Import Tellus modules
from tellus.simulation.simulation import Simulation
from tellus.location.location import Location, LocationKind
from tellus.simulation.context import LocationContext

print("✅ Tellus modules imported successfully")

Let's create a temporary directory structure for our examples:

In [ ]:
# Create temporary directories for our example
temp_dir = Path(tempfile.mkdtemp())
data_dir = temp_dir / "example_data"
output_dir = temp_dir / "outputs"

# Create directory structure
data_dir.mkdir(parents=True)
output_dir.mkdir(parents=True)

# Create some sample files
sample_files = [
    "model_output_day1.nc",
    "model_output_day2.nc", 
    "model_output_day3.nc",
    "metadata.yaml",
    "logs/run.log"
]

for file_path in sample_files:
    full_path = data_dir / file_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Create sample content
    if file_path.endswith('.nc'):
        content = f"# NetCDF data file: {file_path}\n# Generated for Tellus example\n"
    elif file_path.endswith('.yaml'):
        content = "experiment: climate_model_run\nversion: 1.0\nstart_date: 2024-01-01\n"
    else:
        content = f"Log entry for {file_path}\nTimestamp: 2024-01-01 12:00:00\n"
    
    full_path.write_text(content)

print(f"📁 Created example data in: {temp_dir}")
print(f"📊 Sample files: {list(data_dir.rglob('*'))}")

## Working with Simulations

Simulations in Tellus represent computational experiments or datasets. Let's create our first simulation:

In [ ]:
# Create a new simulation
sim = Simulation(
    simulation_id="climate-model-example",
    path=str(output_dir)
)

# Add some metadata
sim.attrs.update({
    "model": "CESM2",
    "experiment": "historical", 
    "resolution": "1deg",
    "start_year": 2020,
    "end_year": 2024
})

print(f"🎯 Created simulation: {sim.simulation_id}")
print(f"📂 Base path: {sim.path}")
print(f"🏷️  Attributes: {sim.attrs}")

## Creating Storage Locations

Locations define where data is stored. Let's create a local filesystem location:

In [ ]:
# Create a local storage location
location = Location(
    name="example-data-store",
    kinds=[LocationKind.DISK],
    config={
        "protocol": "file",
        "storage_options": {
            "path": str(data_dir)
        }
    }
)

print(f"💾 Created location: {location.name}")
print(f"🔧 Protocol: {location.config['protocol']}")
print(f"📍 Path: {location.config['storage_options']['path']}")

## Adding Locations to Simulations

Now let's connect our location to the simulation with some context:

In [ ]:
# Create context for path templating
context = LocationContext(
    path_prefix="{{model}}/{{experiment}}",
    metadata={
        "description": "Primary data storage for climate model outputs",
        "access_level": "read-only"
    }
)

# Add location to simulation
sim.add_location(location, context=context)

print(f"🔗 Added location '{location.name}' to simulation '{sim.simulation_id}'")

# Show resolved path
resolved_path = sim.get_location_path(location.name)
print(f"📁 Resolved path: {resolved_path}")

## Listing Files

Let's explore what files are available in our location:

In [ ]:
# Get the location data
loc_data = sim.locations[location.name]
fs = loc_data["location"].fs

# List files at the resolved path
base_path = sim.get_location_path(location.name)
print(f"📋 Files in {base_path}:")

try:
    files = fs.ls(base_path, detail=True)
    for file_info in files:
        name = file_info.get('name', file_info)
        size = file_info.get('size', 'unknown')
        file_type = 'dir' if file_info.get('type') == 'directory' else 'file'
        print(f"  📄 {os.path.basename(name)} ({file_type}, {size} bytes)")
except Exception as e:
    print(f"⚠️  Could not list files: {e}")
    # Fallback to simple directory listing
    actual_path = Path(location.config['storage_options']['path'])
    print(f"📋 Files in {actual_path}:")
    for file_path in actual_path.rglob('*'):
        if file_path.is_file():
            rel_path = file_path.relative_to(actual_path)
            size = file_path.stat().st_size
            print(f"  📄 {rel_path} (file, {size} bytes)")

## File Operations

Let's demonstrate how to work with files using Tellus:

In [ ]:
# Create a destination for downloaded files
download_dir = temp_dir / "downloads"
download_dir.mkdir(exist_ok=True)

# Get the location object
location_obj = sim.locations[location.name]["location"]

# Try to "download" a file (copy from our example data)
try:
    # List available files first
    source_files = list(data_dir.glob('*.nc'))
    if source_files:
        source_file = source_files[0]
        dest_file = download_dir / source_file.name
        
        # Simple file copy (simulating download)
        import shutil
        shutil.copy2(source_file, dest_file)
        
        print(f"⬇️  Downloaded: {source_file.name}")
        print(f"📁 Saved to: {dest_file}")
        print(f"📊 File size: {dest_file.stat().st_size} bytes")
    else:
        print("⚠️  No .nc files found to download")
        
except Exception as e:
    print(f"❌ Download failed: {e}")

## Working with Multiple Files

Tellus can handle batch operations on multiple files:

In [ ]:
# Find and copy multiple files matching a pattern
pattern = "*.nc"
matching_files = list(data_dir.glob(pattern))

print(f"🔍 Found {len(matching_files)} files matching '{pattern}':")

# Simulate batch download with progress
for i, source_file in enumerate(matching_files, 1):
    dest_file = download_dir / f"batch_{source_file.name}"
    
    # Copy file
    shutil.copy2(source_file, dest_file)
    
    # Show progress
    progress = (i / len(matching_files)) * 100
    print(f"  ⬇️  [{progress:5.1f}%] {source_file.name} → {dest_file.name}")

print(f"✅ Batch download complete! Downloaded {len(matching_files)} files.")

## Simulation Management

Let's explore more simulation management features:

In [ ]:
# Display simulation summary
print(f"📊 Simulation Summary")
print(f"   ID: {sim.simulation_id}")
print(f"   Path: {sim.path}")
print(f"   Locations: {len(sim.locations)}")
print(f"   Attributes: {len(sim.attrs)}")

print("\n🏷️  Attributes:")
for key, value in sim.attrs.items():
    print(f"   {key}: {value}")

print("\n📍 Locations:")
for loc_name, loc_info in sim.locations.items():
    location_obj = loc_info["location"]
    context_obj = loc_info.get("context")
    resolved_path = sim.get_location_path(loc_name)
    
    print(f"   {loc_name}:")
    print(f"     Type: {location_obj.kinds[0].name}")
    print(f"     Protocol: {location_obj.config.get('protocol', 'unknown')}")
    print(f"     Resolved Path: {resolved_path}")
    
    if context_obj and context_obj.metadata:
        print(f"     Metadata: {context_obj.metadata}")

## Path Templating

One of Tellus's powerful features is path templating. Let's see how it works:

In [ ]:
# Create a new location with more complex templating
archive_location = Location(
    name="archive-storage",
    kinds=[LocationKind.ARCHIVE],
    config={
        "protocol": "file",
        "storage_options": {
            "path": str(temp_dir / "archive")
        }
    }
)

# Create archive directory
(temp_dir / "archive").mkdir(exist_ok=True)

# Add with complex path template
archive_context = LocationContext(
    path_prefix="{{model}}/{{experiment}}/{{resolution}}/year_{{start_year}}-{{end_year}}",
    metadata={
        "description": "Long-term archive storage",
        "retention_policy": "10 years"
    }
)

sim.add_location(archive_location, context=archive_context)

# Show how the path is resolved
archive_path = sim.get_location_path(archive_location.name)
print(f"🗂️  Archive location added")
print(f"   Template: {{model}}/{{experiment}}/{{resolution}}/year_{{start_year}}-{{end_year}}")
print(f"   Resolved: {archive_path}")
print(f"   Full path: {Path(archive_location.config['storage_options']['path']) / archive_path}")

## Cleanup

Let's clean up our temporary files:

In [ ]:
# Clean up temporary directory
import shutil

try:
    shutil.rmtree(temp_dir)
    print(f"🧹 Cleaned up temporary directory: {temp_dir}")
except Exception as e:
    print(f"⚠️  Could not clean up {temp_dir}: {e}")

## Summary

In this tutorial, you learned:

✅ **Simulation Management**: How to create simulations with metadata and attributes  
✅ **Location Configuration**: Setting up storage locations with different protocols  
✅ **Context and Templating**: Using path templates with simulation attributes  
✅ **File Operations**: Transferring individual and multiple files  
✅ **Path Resolution**: How Tellus resolves templated paths dynamically  

## Next Steps

- Try the {doc}`remote-data` example to work with SSH and cloud storage
- Explore {doc}`workflow-integration` to use Tellus with Snakemake
- Check out the {doc}`../user-guide/index` for more detailed information

## Real-World Usage

In practice, you would typically:

1. **Use the CLI**: `tellus simulation create`, `tellus location add`, etc.
2. **Configure locations once**: Store authentication and paths in config files
3. **Automate with scripts**: Use Tellus in shell scripts or workflow systems
4. **Monitor progress**: Take advantage of built-in progress bars for large transfers

The Python API shown here is also available for advanced scripting and integration with other tools.